<a href="https://colab.research.google.com/github/reyneej/portofolio2025/blob/main/market_base_datmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab as gc_
    print("Running the code in Google Colab.", gc_) #IN_COLAB = True
    print("Installing required Module, please wait ... ")
    !pip install mlxtend
    !pip install pycaret
except:
    print("Running the code locally, make sure to install required modules.") #IN_COLAB = False

Running the code in Google Colab. <module 'google.colab' from '/usr/local/lib/python3.7/dist-packages/google/colab/__init__.py'>
Installing required Module, please wait ... 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.2 MB/s 
     |████████████████████████████████| 167 kB 41.6 MB/s 
     |████████████████████████████████| 636 kB 45.0 MB/s 
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
     |████████████████████████████████| 88 kB 6.7 MB/s 
     |████████████████████████████████| 315 kB 53.6 MB/s 
     |████████████████████████████████| 6.8 MB 45.3 MB/s 
     |████████████████████████████████| 1.7 MB 75.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... don

In [ ]:
# Import Module
import warnings; warnings.simplefilter('ignore')
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from itertools import combinations
from collections import Counter
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pycaret.arules import *

%matplotlib inline
plt.style.use('bmh'); sns.set()

DATASET

In [ ]:
# Import Data Set
file_ = 'data/Transaksi.csv'
try: # Running Locally
    customer = pd.read_csv(file_, low_memory = False, encoding = 'utf8')
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/frianlh/DataMining-Project/main/data/Transaksi.csv
    transaksi = pd.read_csv(file_, low_memory = False, encoding = 'utf8')

--2022-10-21 06:15:44--  https://raw.githubusercontent.com/frianlh/DataMining-Project/main/data/Transaksi.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1741 (1.7K) [text/plain]
Saving to: ‘data/Transaksi.csv’

Transaksi.csv       100%[===================>]   1.70K  --.-KB/s    in 0s      

2022-10-21 06:15:44 (33.0 MB/s) - ‘data/Transaksi.csv’ saved [1741/1741]



In [ ]:
# Show The First Five Data
transaksi.head()

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
0,1.0,01/07/2022 0:00,Mr. A,1,Roti,Sri Roti,2
1,NaN,NaN,NaN,2,Air Minum,Aquos,1
2,NaN,NaN,NaN,11,Mie Instan,Indomi,2
3,NaN,NaN,NaN,12,Mie Instan,Mi Sedap,1
4,2.0,01/07/2022 0:00,Mr. B,2,Air Minum,Aquos,1


In [ ]:
# Show The Last Five Data
transaksi.tail()

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
51,NaN,NaN,NaN,2,Air Minum,Aquos,1
52,NaN,NaN,NaN,4,Sabun,Lifboy,1
53,NaN,NaN,NaN,6,Snack,Chiki,2
54,18.0,02/10/2022 0:00,Mr. B,1,Roti,Sri Roti,2
55,NaN,NaN,NaN,10,Air Minum,Sprit,2


In [ ]:
# Data Size
N, P = transaksi.shape
print('baris = ', N, ', Kolom (jumlah variabel) = ', P)

# Variable Type
print("Tipe Variabel df = ", type(transaksi))

baris =  56 , Kolom (jumlah variabel) =  7
Tipe Variabel df =  <class 'pandas.core.frame.DataFrame'>


In [ ]:
# Data Information
transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID Transaksi  18 non-null     float64
 1   Tanggal       18 non-null     object 
 2   Konsumen      18 non-null     object 
 3   ID Produk     56 non-null     int64  
 4   Nama Produk   56 non-null     object 
 5   Merek         56 non-null     object 
 6   Jumlah        56 non-null     int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 3.2+ KB


**Pre** **Processing**

In [ ]:
# Copy Data
df = transaksi.copy()

In [ ]:
# Input data NaN in column ID Transaksi, Tanggal, and Konsumen
df.ffill(axis = 0, inplace = True)
df.head()

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
0,1.0,01/07/2022 0:00,Mr. A,1,Roti,Sri Roti,2
1,1.0,01/07/2022 0:00,Mr. A,2,Air Minum,Aquos,1
2,1.0,01/07/2022 0:00,Mr. A,11,Mie Instan,Indomi,2
3,1.0,01/07/2022 0:00,Mr. A,12,Mie Instan,Mi Sedap,1
4,2.0,01/07/2022 0:00,Mr. B,2,Air Minum,Aquos,1


In [ ]:
# Formating Type Data Object in Variabel Tanggal to Date
df["Tanggal"] = pd.to_datetime(df["Tanggal"])
df.head()

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
0,1.0,2022-01-07,Mr. A,1,Roti,Sri Roti,2
1,1.0,2022-01-07,Mr. A,2,Air Minum,Aquos,1
2,1.0,2022-01-07,Mr. A,11,Mie Instan,Indomi,2
3,1.0,2022-01-07,Mr. A,12,Mie Instan,Mi Sedap,1
4,2.0,2022-01-07,Mr. B,2,Air Minum,Aquos,1


In [ ]:
# Add Column Tahun, Bulan, dan Hari
df["Bulan"] = df["Tanggal"].dt.month_name()
df["Hari"] = df["Tanggal"].dt.day_name()
df.head()

,ID Transaksi,Tanggal,Konsumen,ID Produk,Nama Produk,Merek,Jumlah,Bulan,Hari
0,1.0,2022-01-07,Mr. A,1,Roti,Sri Roti,2,January,Friday
1,1.0,2022-01-07,Mr. A,2,Air Minum,Aquos,1,January,Friday
2,1.0,2022-01-07,Mr. A,11,Mie Instan,Indomi,2,January,Friday
3,1.0,2022-01-07,Mr. A,12,Mie Instan,Mi Sedap,1,January,Friday
4,2.0,2022-01-07,Mr. B,2,Air Minum,Aquos,1,January,Friday


In [ ]:
# Get Data Information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID Transaksi  56 non-null     float64       
 1   Tanggal       56 non-null     datetime64[ns]
 2   Konsumen      56 non-null     object        
 3   ID Produk     56 non-null     int64         
 4   Nama Produk   56 non-null     object        
 5   Merek         56 non-null     object        
 6   Jumlah        56 non-null     int64         
 7   Bulan         56 non-null     object        
 8   Hari          56 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 4.1+ KB


In [ ]:
# Correction Data Type
df["ID Transaksi"] = df["ID Transaksi"].astype("int64")
df["Nama Produk"] = df["Nama Produk"].astype("category")
df["Merek"] = df["Merek"].astype("category")
df["Bulan"] = df["Bulan"].astype("category")
df["Hari"] = df["Hari"].astype("category")

Dari hasil association_rules di atas, kita dapat melihat bahwa Sri Roti, Chiki, Lifboy, Aquos adalah item dengan asosiasi tertinggi. Sebab, “lift” pada keempat item ini memiliki nilai tertinggi, yaitu sebesar 9,0.Maka dari itu, keempat item ini cocok untuk dijual bersama.

